In [1]:
import pickle as pkl
from care import Intermediate
from care.crn.utilities.electro import Electron, Proton, Hydroxide, Water
from ase import Atoms

In [2]:
with open('../scripts/C1O2_Cu111_MD/crn.pkl', 'rb') as f:
    crn = pkl.load(f)

In [ ]:
media = "acidic"
is_electr = True

In [ ]:
# def electro_rxn(reaction, media, is_electro):
#         """
#         Set the elementary reaction as an electrochemical reaction
#         """
#         # If the reaction type is C-H or C-O, the reaction is set as electrochemical
#         if reaction.r_type  == "C-H":
#             # Acidic media: Changing the H intermediate to H+, e- as H(e-)
#             if media == "acidic":

# Acidic media

In [ ]:
from care import ElementaryReaction


# Accessing the water gas Intermediate
h2o_gas = [intermediate for intermediate in crn.intermediates.values() if intermediate.formula == 'H2O' and intermediate.phase == 'gas'][0]
oh_code = [intermediate for intermediate in crn.intermediates.values() if intermediate.formula == 'HO'][0].code
surface_inter = Intermediate(code='*', molecule = Atoms(), phase='surf')

for reaction in crn.reactions:
    if reaction.r_type in ("C-H", "O-H"):
        # Finding its index in the CRN
        idx_rxn = [i for i, rxn in enumerate(crn.reactions) if rxn == reaction][0]

        # Replacing [1]YZCKVEUIGOORGS-UHFFFAOYSA-N*(H*) code of the intermediate H* to H(e)
        reactants = [reactant for reactant in reaction.reactants]
        products = [product for product in reaction.products]
        
        new_reactants = []
        for reactant in reactants:
            if reactant.is_surface:
                continue
            if reactant.formula == 'H':
                new_reactants.append(Proton())
                new_reactants.append(Electron())
            else:
                if not reactant.is_surface:
                    new_reactants.append(reactant)
        new_products = []
        for product in products:
            if product.is_surface:
                continue
            if product.formula == 'H':
                new_products.append(Proton())
                new_products.append(Electron())
            else:
                if not product.is_surface:
                    new_products.append(product)
        
        components = [new_reactants, new_products]

        reaction = ElementaryReaction(
            components=components,
            r_type="PCET",
            is_electro=True
        )
        # Updating the CRN with the new reaction
        crn.reactions[idx_rxn] = reaction

    if reaction.r_type in ("C-O", "O-O"):
        # Checking if there is a OH* in the reaction components
        for component_set in reaction.components:
            if oh_code in [component.code for component in component_set]:
                # Finding its index in the CRN
                idx_rxn = [i for i, rxn in enumerate(crn.reactions) if rxn == reaction][0]
                
                # Generating new components
                new_reactants = []
                for reactant in reaction.reactants:
                    if reactant.is_surface:
                        new_reactants.append(Proton())
                        new_reactants.append(Electron())
                    if reactant.formula == 'HO':
                        new_reactants.append(h2o_gas)
                        new_reactants.append(surface_inter)
                    else:
                        if reactant.is_surface:
                            continue
                        new_reactants.append(reactant)
                
                new_products = []
                for product in reaction.products:
                    if product.is_surface:
                        # Adding as much proton and electron as the number of OH* in the reaction
                        new_products.append(Proton())
                        new_products.append(Electron())
                    if product.formula == 'HO':
                        new_products.append(h2o_gas)
                        if len(reaction.products) == 1:
                            new_products.append(surface_inter)
                    else:
                        if product.is_surface:
                            continue
                        new_products.append(product)

                components = [new_reactants, new_products]

                reaction = ElementaryReaction(
                    components=components,
                    r_type="PCET",
                    is_electro=True
                )
                # Updating the CRN with the new reaction
                crn.reactions[idx_rxn] = reaction


In [ ]:
for reaction in crn.reactions:
    if reaction.r_type == 'PCET':
        print(reaction.code, reaction.is_electro)

# Alkaline media

In [4]:
from care import ElementaryReaction


# Accessing the water gas Intermediate
h2o_gas = [intermediate for intermediate in crn.intermediates.values() if intermediate.formula == 'H2O' and intermediate.phase == 'gas'][0]
oh_code = [intermediate for intermediate in crn.intermediates.values() if intermediate.formula == 'HO'][0].code
surface_inter = Intermediate(code='*', molecule = Atoms(), phase='surf')

for reaction in crn.reactions:
    if reaction.r_type in ("C-H", "O-H"):
        # Finding its index in the CRN
        idx_rxn = [i for i, rxn in enumerate(crn.reactions) if rxn == reaction][0]

        # Replacing [1]YZCKVEUIGOORGS-UHFFFAOYSA-N*(H*) code of the intermediate H* to H(e)
        reactants = [reactant for reactant in reaction.reactants]
        products = [product for product in reaction.products]
        
        new_reactants = []
        for reactant in reactants:
            if reactant.is_surface:
                new_reactants.append(Hydroxide())
            if reactant.formula == 'H':
                new_reactants.append(Water())
                new_reactants.append(Electron())
            else:
                if not reactant.is_surface:
                    new_reactants.append(reactant)
        new_products = []
        for product in products:
            if product.is_surface:
                new_reactants.append(Hydroxide())
            if product.formula == 'H':
                new_products.append(Water())
                new_products.append(Electron())
            else:
                if not product.is_surface:
                    new_products.append(product)
        
        components = [new_reactants, new_products]
        reaction = ElementaryReaction(
            components=components,
            r_type="PCET",
            is_electro=True
        )
        # Updating the CRN with the new reaction
        crn.reactions[idx_rxn] = reaction

    if reaction.r_type in ("C-O", "O-O"):
        # Checking if there is a OH* in the reaction components
        for component_set in reaction.components:
            if oh_code in [component.code for component in component_set]:
                # Finding its index in the CRN
                idx_rxn = [i for i, rxn in enumerate(crn.reactions) if rxn == reaction][0]
                
                # Generating new components
                new_reactants = []
                for reactant in reaction.reactants:
                    if reactant.is_surface:
                        new_reactants.append(Water())
                        new_reactants.append(Electron())
                    if reactant.formula == 'HO':
                        new_reactants.append(h2o_gas)
                        new_reactants.append(Hydroxide())
                    else:
                        if reactant.is_surface:
                            continue
                        new_reactants.append(reactant)
                
                new_products = []
                for product in reaction.products:
                    if product.is_surface:
                        # Adding as much proton and electron as the number of OH* in the reaction
                        new_products.append(Water())
                        new_products.append(Electron())
                    if product.formula == 'HO':
                        new_products.append(h2o_gas)
                        new_products.append(Hydroxide())
                        if len(reaction.products) == 1:
                            new_products.append(surface_inter)
                    else:
                        if product.is_surface:
                            continue
                        new_products.append(product)

                components = [new_reactants, new_products]

                reaction = ElementaryReaction(
                    components=components,
                    r_type="PCET",
                    is_electro=True
                )
                # Updating the CRN with the new reaction
                crn.reactions[idx_rxn] = reaction

[1]*+[1]CFHIDWOYWUOIHU-UHFFFAOYSA-N*(CHO*)<->[1]UGFAIRIUMAVXCW-UHFFFAOYSA-N*(CO*)+[1]YZCKVEUIGOORGS-UHFFFAOYSA-N*(H*)
[[OH-(OH-(g)), CFHIDWOYWUOIHU-UHFFFAOYSA-N*(CHO*)], [UGFAIRIUMAVXCW-UHFFFAOYSA-N*(CO*), H2O(H2O(g)), e-(e-(g))]]
[1]*+[1]BQIKRTGTPBOIMK-UHFFFAOYSA-N*(CH2O2*)<->[1]YZCKVEUIGOORGS-UHFFFAOYSA-N*(H*)+[1]JRCLCXAVSVCEQC-UHFFFAOYSA-N*(CHO2*)
[[OH-(OH-(g)), BQIKRTGTPBOIMK-UHFFFAOYSA-N*(CH2O2*)], [H2O(H2O(g)), e-(e-(g)), JRCLCXAVSVCEQC-UHFFFAOYSA-N*(CHO2*)]]
[1]OXOJDNUSDILZJY-UHFFFAOYSA-N*(CH3O2*)+[1]*<->[1]TUJKJAMUKRIRHC-UHFFFAOYSA-N*(HO*)+[1]WSFSSNUMVMOOMR-UHFFFAOYSA-N*(CH2O*)
[[OXOJDNUSDILZJY-UHFFFAOYSA-N*(CH3O2*), H2O(H2O(g)), e-(e-(g))], [XLYOFNOQVPJJNP-UHFFFAOYSA-Ng(H2O(g)), OH-(OH-(g)), WSFSSNUMVMOOMR-UHFFFAOYSA-N*(CH2O*)]]
[1]WCYWZMWISLQXQU-UHFFFAOYSA-N*(CH3*)+[1]*<->[1]HZVOZRGWRWCICA-UHFFFAOYSA-N*(CH2*)+[1]YZCKVEUIGOORGS-UHFFFAOYSA-N*(H*)
[[WCYWZMWISLQXQU-UHFFFAOYSA-N*(CH3*), OH-(OH-(g))], [HZVOZRGWRWCICA-UHFFFAOYSA-N*(CH2*), H2O(H2O(g)), e-(e-(g))]]
[1]CKFGINPQOCXMAZ-U

In [6]:
for reaction in crn.reactions:
    if reaction.r_type == 'PCET':
        print(reaction.code)

[1]OKKJLVBELUTLKV-UHFFFAOYSA-N*(CH4O*)+[1]Water(H2O)+[1]Electron(e-)<->[1]XLYOFNOQVPJJNP-UHFFFAOYSA-Ng(H2O(g))+[1]Hydroxide(OH-)+[1]WCYWZMWISLQXQU-UHFFFAOYSA-N*(CH3*)
[1]Hydroxide(OH-)+[1]CFHIDWOYWUOIHU-UHFFFAOYSA-N*(CHO*)<->[1]Water(H2O)+[1]UGFAIRIUMAVXCW-UHFFFAOYSA-N*(CO*)+[1]Electron(e-)
[1]Hydroxide(OH-)+[1]BQIKRTGTPBOIMK-UHFFFAOYSA-N*(CH2O2*)<->[1]Water(H2O)+[1]JRCLCXAVSVCEQC-UHFFFAOYSA-N*(CHO2*)+[1]Electron(e-)
[1]Water(H2O)+[1]OXOJDNUSDILZJY-UHFFFAOYSA-N*(CH3O2*)+[1]Electron(e-)<->[1]XLYOFNOQVPJJNP-UHFFFAOYSA-Ng(H2O(g))+[1]Hydroxide(OH-)+[1]WSFSSNUMVMOOMR-UHFFFAOYSA-N*(CH2O*)
[1]Hydroxide(OH-)+[1]WCYWZMWISLQXQU-UHFFFAOYSA-N*(CH3*)<->[1]Water(H2O)+[1]HZVOZRGWRWCICA-UHFFFAOYSA-N*(CH2*)+[1]Electron(e-)
[1]CKFGINPQOCXMAZ-UHFFFAOYSA-N*(CH4O2*)+[1]Hydroxide(OH-)<->[1]Water(H2O)+[1]HBIIMURDAQGNHQ-UHFFFAOYSA-N*(CH3O2*)+[1]Electron(e-)
[1]GRVDJDISBSALJP-UHFFFAOYSA-N*(CH3O*)+[1]Hydroxide(OH-)<->[1]Water(H2O)+[1]WSFSSNUMVMOOMR-UHFFFAOYSA-N*(CH2O*)+[1]Electron(e-)
[1]Hydroxide(OH-)+[1]HZVOZ

In [7]:
for intermediate in crn.intermediates.values():
    print(intermediate.code, intermediate.ads_configs)

VNWKTOKETHGBQD-UHFFFAOYSA-N* {'dft': {'conf': Atoms(symbols='Cu48CH4', pbc=True, cell=[[8.90028074, 0.0, 0.0], [4.45014037, 7.70786922, 0.0], [0.0, 0.0, 20.0]], constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]), calculator=SinglePointCalculator(...)), 'pyg': Data(x=[15, 20], edge_index=[2, 58], edge_attr=[58, 1], elem=[15], node_feats=[21], formula='CH4Cu48'), 'mu': -24.140601479999987, 's': 0}}
VNWKTOKETHGBQD-UHFFFAOYSA-Ng {'gas': {'config': Atoms(symbols='CH4', pbc=True), 'pyg': Data(x=[5, 20], edge_index=[2, 8], edge_attr=[8, 1], elem=[5], node_feats=[21], formula='CH4'), 'mu': -23.780370712280273, 's': 0.37976258993148804}}
GRVDJDISBSALJP-UHFFFAOYSA-N* {'dft': {'conf': Atoms(symbols='Cu48CH3O', pbc=True, cell=[[8.90028074, 0.0, 0.0], [4.45014037, 7.70786922, 0.0], [0.0, 0.0, 20.0]], constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]), calculator=SinglePo

In [8]:
print(type(Electron()))

<class 'care.crn.utilities.electro.Electron'>


In [10]:
a = Electron()
b = Proton()
list_ab = [a, b]

for i in list_ab:
    if isinstance(i, Electron):
        print(i)

Electron(e-)
